In [3]:
!pip install transformers datasets[audio] SentencePiece  soundfile speechbrain accelerate

from huggingface_hub import notebook_login
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, Seq2SeqTrainingArguments, Seq2SeqTrainer, SpeechT5HifiGan, AutoTokenizer
from datasets import load_dataset, DatasetDict, Audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Notebook login

In [ ]:
notebook_login()

Pre-processing

In [10]:
checkpoint = "microsoft/speecht5_tts"
model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)
processor = SpeechT5Processor.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Dataset processing

In [23]:
dataset = load_dataset("mozilla-foundation/common_voice_13_0", "ur", split='train')
dataset[0]

FileNotFoundError: ignored

In [ ]:
dataset

In [ ]:
# # Define a function to resample the audio data to the desired sampling rate (e.g., 16k)
# def resample_audio(example):
#     audio, _ = librosa.load(example["path"], sr=16000)
#     return {**example, "audio": audio}

# # Apply the resampling function to the "audio" column for all splits
# for split in dataset.keys():
#     dataset[split] = dataset[split].map(resample_audio)

In [ ]:
# dataset = dataset.map(lambda example: {"audio": (example["audio"], 16000)}, remove_columns=["audio"])

In [ ]:
# Unable to change the sampling_rate = 16000
# Tried different methods but unable to set the value
# This can affect the training of our model

# This method is not working
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
dataset[0]

In [12]:
# # Remove specific subsets
# subsets_to_remove = ["test", "other", "invalidated"]
# for subset in subsets_to_remove:
#     dataset.pop(subset, None)

Text cleanup for SpeechT5 tokenization

In [13]:
tokenizer = processor.tokenizer

In [14]:
buck2uni = {
            u"\u0627":"A",
            u"\u0627":"A",
            u"\u0675":"A",
            u"\u0673":"A",
            u"\u0630":"A",
            u"\u0622":"AA",
            u"\u0628":"B",
            u"\u067E":"P",
            u"\u062A":"T",
            u"\u0637":"T",
            u"\u0679":"T",
            u"\u062C":"J",
            u"\u0633":"S",
            u"\u062B":"S",
            u"\u0635":"S",
            u"\u0686":"CH",
            u"\u062D":"H",
            u"\u0647":"H",
            u"\u0629":"H",
            u"\u06DF":"H",
            u"\u062E":"KH",
            u"\u062F":"D",
            u"\u0688":"D",
            u"\u0630":"Z",
            u"\u0632":"Z",
            u"\u0636":"Z",
            u"\u0638":"Z",
            u"\u068E":"Z",
            u"\u0631":"R",
            u"\u0691":"R",
            u"\u0634":"SH",
            u"\u063A":"GH",
            u"\u0641":"F",
            u"\u06A9":"K",
            u"\u0642":"K",
            u"\u06AF":"G",
            u"\u0644":"L",
            u"\u0645":"M",
            u"\u0646":"N",
            u"\u06BA":"N",
            u"\u0648":"O",
            u"\u0649":"Y",
            u"\u0626":"Y",
            u"\u06CC":"Y",
            u"\u06D2":"E",
            u"\u06C1":"H",
            u"\u064A":"E"  ,
            u"\u06C2":"AH"  ,
            u"\u06BE":"H"  ,
            u"\u0639":"A"  ,
            u"\u0643":"K" ,
            u"\u0621":"A",
            u"\u0624":"O",
            u"\u0648":"U",
            u"\u060C":"" #seperator ulta comma
}
def transString(string, reverse=0):
    '''Given a Unicode string, transliterate into Buckwalter. To go from
    Buckwalter back to Unicode, set reverse=1'''
    for k, v in buck2uni.items():
      if not reverse:
            string = string.replace(k, v)
      else:
            string = string.replace(v, k)
    return string
print(dataset[0]["sentence"])
print(transString(dataset[0]["sentence"]))

NameError: ignored

In [15]:
def prepare_dataset(batch):
  """Function to preprocess the dataset with the .map method"""
  transcription = transString(batch["sentence"])

  if transcription.startswith('"') and transcription.endswith('"'):
    # we can remove trailing quotation marks as they do not affect the transcription
    transcription = transcription[1:-1]

  if transcription[-1] not in [".", "?", "!"]:
    # append a full-stop to sentences that do not end in punctuation
    transcription = transcription + "."

  batch["sentence"] = transcription

  return batch

dataset = dataset.map(prepare_dataset, desc="preprocess dataset")
dataset

NameError: ignored

Replacing characters that are unknown in the to be trained language

In [16]:
dataset[0]

NameError: ignored

Speaker embeddings

In [17]:
import torch
from speechbrain.pretrained import EncoderClassifier
import os

In [18]:
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

Processing the dataset

In [19]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["sentence"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

Mapping the dataset

In [22]:
dataset = dataset.map(prepare_dataset)
dataset

NameError: ignored

In [21]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200


dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

NameError: ignored

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant', 'input_ids', 'labels', 'speaker_embeddings'],
        num_rows: 3716
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant', 'input_ids', 'labels', 'speaker_embeddings'],
        num_rows: 413
    })
})

Data collator

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

In [ ]:
@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

Traing the model

In [ ]:
from functools import partial

In [ ]:
# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

Training arguments

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="urdu_tts_finetuned_voxpopuli_nl",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

Trainer

In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

Cloning https://huggingface.co/ahmadaarif/urdu_tts_finetuned_voxpopuli_nl into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.551800,0.500076
2000,0.521300,0.491691
3000,0.509100,0.482467
4000,0.505300,0.483712


TrainOutput(global_step=4000, training_loss=0.5511965277194977, metrics={'train_runtime': 6615.8383, 'train_samples_per_second': 19.348, 'train_steps_per_second': 0.605, 'total_flos': 6770467937322144.0, 'train_loss': 0.5511965277194977, 'epoch': 34.45})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/558M [00:00<?, ?B/s]

Upload file runs/Jul17_12-23-16_e19fb033b78a/events.out.tfevents.1689596659.e19fb033b78a.2966.0:   0%|        …

To https://huggingface.co/ahmadaarif/urdu_tts_finetuned_voxpopuli_nl
   1142bc1..2001a92  main -> main

   1142bc1..2001a92  main -> main

To https://huggingface.co/ahmadaarif/urdu_tts_finetuned_voxpopuli_nl
   2001a92..7dc11e2  main -> main

   2001a92..7dc11e2  main -> main



'https://huggingface.co/ahmadaarif/urdu_tts_finetuned_voxpopuli_nl/commit/2001a929c484e409cc88a4d36f44a221312ca6e4'

Inference

In [4]:
model = SpeechT5ForTextToSpeech.from_pretrained(
    "ahmadaarif/urdu_tts_finetuned_voxpopuli_nl"
)

In [7]:
example = dataset["test"][304]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

NameError: ignored

In [8]:
text = "MRA NAM AHMAD HA"

In [9]:
inputs = processor(text=text, return_tensors="pt")

NameError: ignored

In [ ]:
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

Audio Inference

In [6]:
from IPython.display import Audio
import numpy as np

In [ ]:
Audio(speech.numpy(), rate=16000)

Interface

In [ ]:
!pip install gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 6.5 MB/s eta 0:00:00
  

In [1]:
def predict(input_text):
  output = trainer.predict(input_text)
  return output

In [2]:
interface = gr.Interface(fn=predict, inputs="text", outputs="audio", verbose = True, title="Text to Speech for Urdu",
                         description = "An Urdu Text to Speech application. It can produce random noise  as well as the accuracy is around 47%. It can be fine-tuned further as well. Right now it works successfully on very basic urdu text, such the ones in the example.")
interface.launch(inline=False, debug=True, share=True)

NameError: ignored